# GPU Energy Modeling Demo

This notebook demonstrates the key components of the GPU energy modeling framework.

In [ ]:
import sys
import os
sys.path.append('.')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from src.benchmarks.compute_benchmarks import MatrixMultiplication, ConvolutionBenchmark
from src.benchmarks.memory_benchmarks import MemoryCopy, RandomAccess
from src.data_collection.collectors import SimulatedPowerCollector, PerformanceCounterCollector
from src.modeling.energy_model import LinearEnergyModel
from src.analysis.visualization import (
    plot_power_over_time, 
    plot_component_breakdown,
    plot_efficiency_comparison,
    plot_model_feature_importance
)
from src.analysis.efficiency import (
    calculate_energy_consumption,
    analyze_energy_efficiency,
    identify_efficiency_bottlenecks,
    what_if_analysis
)

# Create data directories if they don't exist
os.makedirs('data', exist_ok=True)

## 1. Running GPU Benchmarks

First, let's run some benchmarks to generate workloads.

In [ ]:
# Create benchmark instances
matmul_benchmark = MatrixMultiplication()
conv_benchmark = ConvolutionBenchmark()
memcopy_benchmark = MemoryCopy()
random_access_benchmark = RandomAccess()

# Define benchmark parameters
matmul_params = {
    'matrix_size': 1024,
    'dtype': np.float32
}

conv_params = {
    'input_size': 224,
    'kernel_size': 3,
    'channels': 3,
    'filters': 64
}

memcopy_params = {
    'buffer_size_mb': 256,
    'iterations': 5
}

random_access_params = {
    'array_size_mb': 128,
    'access_count': 1000000
}

In [ ]:
# Run the benchmarks
print("Running Matrix Multiplication benchmark...")
matmul_results = matmul_benchmark.run(matmul_params)
print(f"Execution time: {matmul_results['mean_execution_time']:.4f} seconds")

print("\nRunning Convolution benchmark...")
conv_results = conv_benchmark.run(conv_params)
print(f"Execution time: {conv_results['mean_execution_time']:.4f} seconds")

print("\nRunning Memory Copy benchmark...")
memcopy_results = memcopy_benchmark.run(memcopy_params)
print(f"Execution time: {memcopy_results['mean_execution_time']:.4f} seconds")

print("\nRunning Random Access benchmark...")
random_access_results = random_access_benchmark.run(random_access_params)
print(f"Execution time: {random_access_results['mean_execution_time']:.4f} seconds")

## 2. Collecting Power Data

Next, let's collect simulated power data for these benchmarks.

In [ ]:
# Create power collector
power_collector = SimulatedPowerCollector(sampling_interval=0.1)
counter_collector = PerformanceCounterCollector()

# Collect power data for matrix multiplication
print("Collecting power data for Matrix Multiplication...")
# Create an activity pattern to simulate different phases
duration = 5.0  # seconds
num_samples = int(duration / power_collector.sampling_interval)
# Simulate a ramp-up, steady state, and cool-down pattern
activity_pattern = np.concatenate([
    np.linspace(0.2, 0.9, num_samples // 4),  # Ramp up
    np.linspace(0.9, 1.0, num_samples // 4),   # Peak
    np.ones(num_samples // 4) * 0.95,          # Steady state
    np.linspace(0.9, 0.2, num_samples // 4)    # Cool down
])

matmul_power_data = power_collector.collect_for_duration(duration, activity_pattern)
matmul_power_df = pd.DataFrame(matmul_power_data)

# Save data
matmul_power_df.to_csv('data/matmul_power.csv', index=False)
print(f"Collected {len(matmul_power_data)} power samples")

# Collect sample performance counter data
counter_data = []
for i in range(len(matmul_power_data)):
    counter_data.append(counter_collector.collect_counters())
    
counter_df = pd.DataFrame([{
    'timestamp': item['timestamp'],
    **item['counters']
} for item in counter_data])

counter_df.to_csv('data/matmul_counters.csv', index=False)
print(f"Collected {len(counter_data)} counter samples")

## 3. Visualizing Power Data

Let's visualize the power consumption data.

In [ ]:
# Plot power over time
component_cols = ['compute_power', 'memory_power', 'io_power']
fig = plot_power_over_time(
    matmul_power_df, 
    component_cols=component_cols,
    title="Matrix Multiplication Power Consumption",
    save_path="data/matmul_power_over_time.png"
)
plt.show()

# Plot component breakdown
fig = plot_component_breakdown(
    matmul_power_df,
    component_cols=component_cols,
    title="Matrix Multiplication Power Breakdown",
    save_path="data/matmul_power_breakdown.png"
)
plt.show()

## 4. Building an Energy Model

Now, let's build a simple energy model using the collected data.

In [ ]:
# Prepare training data
# For a real application, we would use actual performance counter data
# Here we're using the simulated counter data

# Merge counter data with power data based on closest timestamp
merged_df = pd.merge_asof(
    counter_df.sort_values('timestamp'),
    matmul_power_df[['timestamp', 'total_power']].sort_values('timestamp'),
    on='timestamp',
    direction='nearest'  # Use nearest match to ensure better correspondence
)

print(f"Shape of merged data: {merged_df.shape}")

# Check for data quality
for col in ['sm_activity', 'memory_utilization', 'cache_hit_rate', 'memory_throughput', 'total_power']:
    print(f"{col}: mean={merged_df[col].mean():.2f}, std={merged_df[col].std():.2f}, range={merged_df[col].max() - merged_df[col].min():.2f}")

# Create correlation matrix
correlation = merged_df[['sm_activity', 'memory_utilization', 'cache_hit_rate', 
                        'instructions_executed', 'memory_throughput', 'total_power']].corr()
                        
print("\nCorrelation with total_power:")
for col in ['sm_activity', 'memory_utilization', 'cache_hit_rate', 'instructions_executed', 'memory_throughput']:
    print(f"  {col}: {correlation.loc[col, 'total_power']:.4f}")

# Select features and target
feature_cols = [
    'sm_activity', 'memory_utilization', 'cache_hit_rate', 
    'instructions_executed', 'memory_throughput'
]
X = merged_df[feature_cols].values
y = merged_df['total_power'].values

# Add small amount of noise to ensure model doesn't have perfect fit
# This simulates real-world measurement noise
np.random.seed(42)  # For reproducibility
X_noisy = X + np.random.normal(0, 0.05 * np.mean(X, axis=0), size=X.shape)
y_noisy = y + np.random.normal(0, 0.05 * np.mean(y), size=y.shape)

# Create scatter plots to verify relationships
plt.figure(figsize=(15, 10))
for i, col in enumerate(feature_cols):
    plt.subplot(2, 3, i+1)
    plt.scatter(X_noisy[:, i], y_noisy, alpha=0.5)
    plt.title(f'{col} vs Power')
    plt.xlabel(col)
    plt.ylabel('total_power')

plt.tight_layout()
plt.show()

# Train the model
model = LinearEnergyModel(model_name="gpu_power_model", alpha=0.1)
training_results = model.train(X_noisy, y_noisy)

print("\nModel Training Results:")
print(f"Training RMSE: {training_results['train_metrics']['rmse']:.4f}")
print(f"Validation RMSE: {training_results['val_metrics']['rmse']:.4f}")
print(f"Validation R²: {training_results['val_metrics']['r2']:.4f}")

# Print model coefficients
print("\nModel Coefficients:")
for feature, coef in model.feature_importance.items():
    feature_idx = int(feature.split('_')[1])
    feature_name = feature_cols[feature_idx]
    print(f"  {feature_name}: {coef:.4f}")

In [ ]:
# Visualize feature importance from the noisy model (which should have proper values)
feature_importance_dict = {}
for feature, coef in model.feature_importance.items():
    feature_idx = int(feature.split('_')[1])
    feature_name = feature_cols[feature_idx]
    feature_importance_dict[feature_name] = coef

fig = plot_model_feature_importance(
    feature_importance_dict,
    title="GPU Power Model Feature Importance",
    save_path="data/model_feature_importance.png"
)
plt.show()

## 5. Energy Efficiency Analysis

Let's analyze the energy efficiency of the benchmarks.

In [ ]:
# Calculate energy consumption
total_energy = calculate_energy_consumption(matmul_power_df)
print(f"Total energy consumption: {total_energy:.2f} joules")

# Prepare benchmark results with additional information
matmul_metrics = {
    'operations': matmul_results['raw_results'][0]['operations'],
    'execution_time': matmul_results['mean_execution_time']
}

# Analyze efficiency
efficiency_metrics = analyze_energy_efficiency(matmul_metrics, matmul_power_df)

print("\nEfficiency Metrics:")
for metric, value in efficiency_metrics.items():
    print(f"{metric}: {value:.2f}")

In [ ]:
# Identify potential bottlenecks
bottlenecks = identify_efficiency_bottlenecks(matmul_power_df, counter_df)

print("Efficiency Bottleneck Analysis:")
for bottleneck, details in bottlenecks.items():
    print(f"\n{bottleneck}:")
    if isinstance(details, dict):
        for key, value in details.items():
            print(f"  {key}: {value}")
    else:
        print(f"  {details}")

## 6. What-If Analysis

Let's perform some what-if analysis to identify optimization opportunities.

In [ ]:
# Define baseline and scenario adjustments
baseline_features = np.mean(X, axis=0)  # Average of all observations

# Define scenarios to analyze
scenarios = {
    'Improved_Cache_Hits': {
        'cache_hit_rate': (1.25, 0)  # 25% increase in cache hit rate
    },
    'Reduced_Memory_Bandwidth': {
        'memory_throughput': (0.8, 0)  # 20% decrease in memory bandwidth
    },
    'Optimized_Compute': {
        'instructions_executed': (0.7, 0),  # 30% fewer instructions
        'sm_activity': (0.9, 0)  # 10% decrease in SM activity
    },
    'Combined_Optimizations': {
        'cache_hit_rate': (1.2, 0),  # 20% increase in cache hits
        'instructions_executed': (0.8, 0),  # 20% fewer instructions
        'memory_throughput': (0.9, 0)  # 10% less memory bandwidth
    }
}

# Run what-if analysis
scenario_results = what_if_analysis(model, baseline_features, scenarios, feature_cols)

# Print results
print(f"Baseline Power: {scenario_results['baseline_power']:.2f} W")
print("\nScenario Analysis:")

for scenario, results in scenario_results['scenarios'].items():
    print(f"\n{scenario}:")
    print(f"  Predicted Power: {results['power']:.2f} W")
    print(f"  Change: {results['absolute_change']:.2f} W ({results['percent_change']:.2f}%)")
    
    print("  Adjusted Features:")
    for feature, value in results['adjusted_features'].items():
        print(f"    {feature}: {value:.2f}")

## 7. Conclusion and Next Steps

Based on our analysis, we can identify the following key insights and opportunities for optimization:

1. **Power Breakdown**: The primary power consumers in our benchmark are...
2. **Key Performance Counters**: The most significant predictors of power consumption are...
3. **Optimization Opportunities**: The most promising optimization approaches are...

### Next Steps

- Collect real hardware measurements for more accurate modeling
- Extend the benchmark suite with more diverse workloads
- Implement more sophisticated modeling approaches (ML-based models)
- Develop automated optimization recommendation system
- Create comprehensive visualization dashboard